In [1]:
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def load_slangwords_stopwords():
    global slangwords, stopwords
    
    with open('./slangwords.txt', 'rt') as file_1:
        slangwords = {}
        for line in file_1:
            (key, val) = line.rstrip().split(":")
            slangwords[key] = val

    with open('./stopwords_v2.txt') as file_2:
        stopwords = [item.rstrip() for item in list(file_2)]

load_slangwords_stopwords()        

def preprocess(comment: str):
  
  # Menghilangkan nama saham
  comment = re.sub("\$[A-Z]{4}| [A-Z]{4} ", " ", comment)

  # Mengubah text ke Lowercase agar semua data seragam
  comment = comment.lower()

  # Menghilangkan @/Mention karena pada berita palsu ada mention akun twitter
  comment = re.sub("@[A-Za-z0-9]+", " ", comment)

  # Menghilangkan #/Hashtag untuk mengantisipasi karena berita palsu mengambil dari twitter
  comment = re.sub("#[A-Za-z0-9_]+", " ", comment)
  
  # Menghilangkan \n untuk antisipasi
  comment = re.sub(r"\n", " ",comment)

  # Menghilangkan Whitespace untuk antisipasi
  comment = comment.strip()

  # Menghilangkan Link dikarenakan berita palsu terdapat link ke artikel lain
  comment = re.sub(r"http\S+", " ", comment)
  comment = re.sub(r"www.\S+", " ", comment)

  # Menghilangkan yang Bukan Huruf seperti Emoji, Simbol Matematika (seperti μ), dst untuk antisipasi
  comment = re.sub("[^A-Za-z\s']", " ", comment)

  # Melakukan Tokenisasi
  tokens = word_tokenize(comment)

  # Menghilangkan Stopwords
  comment = ' '.join([word for word in tokens if word not in stopwords])

  # Melakukan lemmatizing
  comment = " ".join(slangwords.get(word, word) for word in comment.split())
  
  return comment

preprocess('@TakaTin sudah ada dong pola kadal purba di $INDF\n\n-------\n$MYOR $ICBP $GOOD $LPPF\n\n⬇️⬇️⬇️\nhttps://stockbit.com/post/8338381')

[nltk_data] Downloading package punkt to /Users/jason/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'dong pola kadal purba'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../ICBP_MLBI_labeled.csv')

X_train, X_test, y_train, y_test = train_test_split(df['comment'], df['label 2'])
isinstance(X_train, pd.Series)

True

In [4]:
import sklearn

sklearn.__version__

'1.0.2'

In [13]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = joblib.load('./count_vect.pkl')

In [14]:
def transform(comment: pd.Series):
    return vectorizer.transform(X_train)

transform(X_train)

<136x5020 sparse matrix of type '<class 'numpy.int64'>'
	with 5087 stored elements in Compressed Sparse Row format>